In [1]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
#from tensorflow.examples.tutorials.mnist import input_data
from torch.autograd import Variable
import sklearn.metrics as metrics
import pickle

In [2]:
path = '/home/daiict/CVShare/Jeni/hts/bioassay-datasets/'
p_fingerprints = []
c_fingerprints = []
labels = []
with open(path+'AID362red_test.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        p_fingerprints.append(row[:112])
        c_fingerprints.append(row[112:-1])
        labels.append(row[-1])

In [3]:
p_fingerprints = np.asarray(p_fingerprints)[1:]
p_fingerprints = p_fingerprints.astype(int)
c_fingerprints = np.asarray(c_fingerprints)[1:]
c_fingerprints = c_fingerprints.astype(float)
#p2_fingerprints = np.ones(p_fingerprints.shape)
(no_examples , ip_dim) = p_fingerprints.shape
ip_dim2 = c_fingerprints.shape[1]

labels = labels[1:]
print(no_examples,ip_dim)
print(ip_dim2)
p_fingerprints[(p_fingerprints==0)] = -1

(856, 112)
32


In [4]:
labels2 = np.zeros((len(labels),1))
for i,l in enumerate(labels):
    if l=='Active':
        labels2[i] = 1
    else:
        labels2[i] = 0
labels2 = labels2.astype(int)

In [5]:
class mlp(nn.Module):
    def __init__(self):
        super(mlp,self).__init__()
        self.l1 = nn.Linear(ip_dim,1000)
        self.l2 = nn.Linear(1000,500)
        self.l3 = nn.Linear(500,200)
        self.l4 = nn.Linear(200,2)
        
    def forward(self,x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = (self.l4(x))
        
        return x

In [6]:
class c_mlp(nn.Module):
    def __init__(self):
        super(c_mlp,self).__init__()
        self.l1 = nn.Linear(ip_dim2,100)
        self.l2 = nn.Linear(100,100)
        self.l3 = nn.Linear(100,100)
        self.l4 = nn.Linear(100,50)
        self.l5 = nn.Linear(50,2)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        x = F.leaky_relu(self.l3(x))
        x = F.leaky_relu(self.l4(x))
        x = (self.l5(x))
        
        return x

In [7]:
en_predictions = []
c_models = 4
for i in range(1,c_models):
    my_cmlp = c_mlp().cuda()
    path = os.getcwd() + '/model' + str(i)
    my_cmlp.load_state_dict(torch.load(path))
    tmp_op = my_cmlp(Variable(torch.cuda.FloatTensor(c_fingerprints)))
    tmp_op = tmp_op.cpu().data.numpy()
    pred_labels = np.argmax(tmp_op,axis=1)
    en_predictions.append(pred_labels)

In [8]:
b_models = 3
for i in range(b_models):
    my_mlp = mlp().cuda()
    path = os.getcwd() + '/b_model' + str(i)
    my_mlp.load_state_dict(torch.load(path))
    tmp_op = my_mlp(Variable(torch.cuda.FloatTensor(p_fingerprints)))
    tmp_op = tmp_op.cpu().data.numpy()
    pred_labels = np.argmax(tmp_op,axis=1)
    en_predictions.append(pred_labels)

In [9]:
en_predictions = np.asarray(en_predictions)
en_sum = np.sum(en_predictions,axis=0)
#print(en_sum)
en_op = np.zeros(en_sum.shape)
en_op[en_sum >= 2] = 1
en_op[en_sum < 2] = 0
#print(en_op)

In [26]:
cf = metrics.confusion_matrix(labels2,en_op).ravel()
print('tn, fp, fn, tp: ',cf)

('tn, fp, fn, tp: ', array([826,  18,   5,   7]))


In [11]:
print(sum(en_sum))

75


In [13]:
#CM with cont features
with open("cnt_test.txt",'rb') as f:
    tmplist = pickle.load(f)

In [14]:
tmplist

[array([3362,   13,    8,   40]),
 array([3359,   16,    4,   44]),
 array([3358,   17,    3,   45]),
 array([3358,   17,    2,   46]),
 array([3352,   23,    4,   44]),
 array([3344,   31,    1,   47]),
 array([3351,   24,    2,   46]),
 array([3345,   30,    3,   45]),
 array([3333,   42,    0,   48]),
 array([3344,   31,    1,   47]),
 array([3318,   57,    1,   47]),
 array([3341,   34,    1,   47]),
 array([3347,   28,    2,   46]),
 array([3334,   41,    0,   48]),
 array([3337,   38,    1,   47]),
 array([3308,   67,    0,   48]),
 array([3324,   51,    0,   48]),
 array([3329,   46,    0,   48]),
 array([3342,   33,    1,   47]),
 array([3334,   41,    0,   48]),
 array([3306,   69,    1,   47]),
 array([3318,   57,    0,   48])]